In [1]:
!pip install pyannote.audio
!pip install torch
!pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:

import json
import os
from pathlib import Path
from dotenv import load_dotenv
import gc
import pprint

import torch
from tqdm import tqdm
from pyannote.audio import Pipeline
from pyannote.audio.pipelines.utils.hook import ProgressHook

load_dotenv()

/home/specc/.cache/pypoetry/virtualenvs/tts-Eh4q6c48-py3.11/lib/python3.11/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/home/specc/.cache/pypoetry/virtualenvs/tts-Eh4q6c48-py3.11/lib/python3.11/site-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


True

In [3]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"

In [4]:
diarization_pipe = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=os.environ.get("HUGGINGFACE_API_KEY"),
)

diarization_pipe.embeddings_batch_size = 1

diarization_pipe.to(torch.device("cuda"))

In [6]:
with ProgressHook() as hook:
    diarization = diarization_pipe("../data/cache/lck_sample_01.mp3", max_speakers=2, hook=hook)

Output()

In [8]:
data = {"lck_sample_01.mp3": []}

for turn, _, speaker in tqdm(diarization.itertracks(yield_label=True)):
    # data["lck_sample_01.mp3"].append({"start": turn.start, "end": turn.end, "speaker": speaker})
    data.get("lck_sample_01.mp3", []).append({"start": turn.start, "end": turn.end, "speaker": speaker})

7it [00:00, 67650.06it/s]


In [9]:
data.get("lck_sample_01.mp3")

[{'start': 0.008488964346349746,
  'end': 17.818336162988118,
  'speaker': 'SPEAKER_00'},
 {'start': 19.09168081494058,
  'end': 24.01528013582343,
  'speaker': 'SPEAKER_00'},
 {'start': 24.660441426146015,
  'end': 28.582342954159593,
  'speaker': 'SPEAKER_00'},
 {'start': 29.56706281833616,
  'end': 30.212224108658745,
  'speaker': 'SPEAKER_00'},
 {'start': 30.840407470288625,
  'end': 44.7962648556876,
  'speaker': 'SPEAKER_00'},
 {'start': 46.511035653650254,
  'end': 51.40067911714771,
  'speaker': 'SPEAKER_00'},
 {'start': 56.44312393887946,
  'end': 59.99151103565365,
  'speaker': 'SPEAKER_00'}]

In [10]:
with ProgressHook() as hook:
    diarization = diarization_pipe("../data/cache/lck_stream_2023_12_22.mp3", max_speakers=2, hook=hook)

Output()

: 

In [ ]:
for turn, _, speaker in tqdm(diarization.itertracks(yield_label=True)):
    data.get("lck_stream_2023_12_22.mp3", []).append({"start": turn.start, "end": turn.end, "speaker": speaker})

In [ ]:
OUT_PATH = Path("data/temp/diarization_data.json")

if OUT_PATH.exists():
    with open(OUT_PATH, "r") as f:
        data = json.load(f)
else:
    existing_data = {}

existing_data.update(data)    

with open(OUT_PATH, "w") as f:
    json.dump(existing_data, f, indent=4)
